In [17]:
import pandas as pd
import re
import time
import os
import io
import base64
import glob
from io import BytesIO
from barcode import Code128
from barcode.writer import ImageWriter

In [18]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 500)

In [19]:
df_abrv = pd.DataFrame({'attribute':['accessories', 'kids', 'pets', 'home & living', 'adult clothes',],
                        'attribute_abrv':['ACC', 'KD','PET','HL', 'ADC']})
df_abrv

,attribute,attribute_abrv
0,accessories,ACC
1,kids,KD
2,pets,PET
3,home & living,HL
4,adult clothes,ADC


In [20]:
freight_codes = input('enter the list of FREIGHT_COD for this pricing caculation')
freight_codes_list = freight_codes.split(',')
freight_codes_list

['FRE_000009']

In [21]:
file = r"G:\\My Drive\\DB\\product\\pricing\\pricing_{val}.xlsx".format(val=freight_codes_list[0])

df = pd.read_excel(file)

df = pd.merge(df,df_abrv,how = 'left',left_on='main_category',right_on='attribute')

df.head()



,Unnamed: 0,date,freight_cod,purchase_id,identity_code,supplier,description,main_category,product_classification,size,...,revenue_60,price_80,revenue_80,price_100,revenue_100,price_120,revenue_120,assigned_price,attribute,attribute_abrv
0,0,2024-10-09 00:00:00,FRE_000009,PUR_00051,PUR_00051-1,Marlin Estevi,dog collar 20 cm color brown leather,pets,collar,30,...,3.917,20.827,5.222,22.839,6.528,24.852,7.834,22.99,pets,PET
1,1,2024-10-09 00:00:00,FRE_000009,PUR_00051,PUR_00051-2,Marlin Estevi,dog collar 20 cm color black,pets,collar,30,...,3.917,20.827,5.222,22.839,6.528,24.852,7.834,22.99,pets,PET
2,2,2024-10-09 00:00:00,FRE_000009,PUR_00051,PUR_00051-3,Marlin Estevi,dog collar 20 cm color coffee,pets,collar,30,...,3.917,20.827,5.222,22.839,6.528,24.852,7.834,22.99,pets,PET
3,3,2024-10-09 00:00:00,FRE_000009,PUR_00051,PUR_00051-4,Marlin Estevi,dog collar 40 cm color brown leather,pets,collar,40,...,5.001,25.843,6.669,28.413,8.336,30.982,10.003,28.99,pets,PET
4,4,2024-10-09 00:00:00,FRE_000009,PUR_00051,PUR_00051-5,Marlin Estevi,dog collar 40 cm color black,pets,collar,40,...,5.001,25.843,6.669,28.413,8.336,30.982,10.003,28.99,pets,PET


In [22]:
df.columns

Index(['Unnamed: 0', 'date', 'freight_cod', 'purchase_id', 'identity_code',
       'supplier', 'description', 'main_category', 'product_classification',
       'size', 'main_color', 'country', 'item_cost_unit_eu', 'units',
       'price_60', 'revenue_60', 'price_80', 'revenue_80', 'price_100',
       'revenue_100', 'price_120', 'revenue_120', 'assigned_price',
       'attribute', 'attribute_abrv'],
      dtype='object')

In [23]:
columns = ['freight_cod','identity_code','supplier','description', 'attribute_abrv', 'product_classification','size','main_color','country','units','item_cost_unit_eu','price_60','price_120','assigned_price']
df = df[columns].copy()
df.fillna('*',inplace=True)
df['size'] = [x.upper() for x in df['size']]
df['country'] = ['PE' if x == 'peru' else 'MEX' for x in df['country']]
df['price_60'] = df['price_60'].round(2)
df['price_120'] = df['price_120'].round(2)
df['assigned_price'] = df['assigned_price'].round(2)
df

,freight_cod,identity_code,supplier,description,attribute_abrv,product_classification,size,main_color,country,units,item_cost_unit_eu,price_60,price_120,assigned_price
0,FRE_000009,PUR_00051-1,Marlin Estevi,dog collar 20 cm color brown leather,PET,collar,30,brown,MEX,7,6.528013,18.81,24.85,22.99
1,FRE_000009,PUR_00051-2,Marlin Estevi,dog collar 20 cm color black,PET,collar,30,black,MEX,7,6.528013,18.81,24.85,22.99
2,FRE_000009,PUR_00051-3,Marlin Estevi,dog collar 20 cm color coffee,PET,collar,30,coffee,MEX,6,6.528013,18.81,24.85,22.99
3,FRE_000009,PUR_00051-4,Marlin Estevi,dog collar 40 cm color brown leather,PET,collar,40,brown,MEX,16,8.335770,23.27,30.98,28.99
4,FRE_000009,PUR_00051-5,Marlin Estevi,dog collar 40 cm color black,PET,collar,40,black,MEX,15,8.335770,23.27,30.98,28.99
5,FRE_000009,PUR_00051-6,Marlin Estevi,dog collar 40 cm color coffee,PET,collar,40,coffee,MEX,15,8.335770,23.27,30.98,28.99
6,FRE_000009,PUR_00051-7,Marlin Estevi,dog collar 50 cm color brown leather,PET,collar,50,brown,MEX,10,12.754733,34.17,45.97,35.99
7,FRE_000009,PUR_00051-8,Marlin Estevi,dog collar 50 cm color black,PET,collar,50,black,MEX,10,12.754733,34.17,45.97,35.99
8,FRE_000009,PUR_00051-9,Marlin Estevi,dog collar 50 cm color coffee,PET,collar,50,coffee,MEX,10,12.754733,34.17,45.97,35.99
9,FRE_000009,PUR_00052-1,Briseyda,toy in felt dino long neck,KD,toy,--,--,MEX,3,11.549561,31.20,41.88,28.99


In [24]:
df.columns

Index(['freight_cod', 'identity_code', 'supplier', 'description',
       'attribute_abrv', 'product_classification', 'size', 'main_color',
       'country', 'units', 'item_cost_unit_eu', 'price_60', 'price_120',
       'assigned_price'],
      dtype='object')

In [25]:
df['prices'] = [str(a)+' | '+str(b)+' | '+str(c) for a,b,c in zip(df['price_60'], df['price_120'], df['assigned_price'])]
df.drop(columns=['price_60','price_120','assigned_price'],inplace=True)
df.rename(columns={'item_cost_unit_eu':'IC'},inplace='True')
df.rename(columns={'product_classification':'PC'},inplace='True')
df.rename(columns={'attribute_abrv':'Pctg'},inplace='True')
df.rename(columns={'main_color':'clr'},inplace='True')
df['IC'] = [round(x,2) for x in df['IC']]
df.head()

,freight_cod,identity_code,supplier,description,Pctg,PC,size,clr,country,units,IC,prices
0,FRE_000009,PUR_00051-1,Marlin Estevi,dog collar 20 cm color brown leather,PET,collar,30,brown,MEX,7,6.53,18.81 | 24.85 | 22.99
1,FRE_000009,PUR_00051-2,Marlin Estevi,dog collar 20 cm color black,PET,collar,30,black,MEX,7,6.53,18.81 | 24.85 | 22.99
2,FRE_000009,PUR_00051-3,Marlin Estevi,dog collar 20 cm color coffee,PET,collar,30,coffee,MEX,6,6.53,18.81 | 24.85 | 22.99
3,FRE_000009,PUR_00051-4,Marlin Estevi,dog collar 40 cm color brown leather,PET,collar,40,brown,MEX,16,8.34,23.27 | 30.98 | 28.99
4,FRE_000009,PUR_00051-5,Marlin Estevi,dog collar 40 cm color black,PET,collar,40,black,MEX,15,8.34,23.27 | 30.98 | 28.99


In [26]:
df = df.sort_values(by=['Pctg','freight_cod','supplier'],ascending=False)
df = df[['Pctg','freight_cod', 'supplier', 'identity_code','description', 'PC',
       'size', 'clr', 'country', 'units', 'IC', 'prices']]
df.head(60)

,Pctg,freight_cod,supplier,identity_code,description,PC,size,clr,country,units,IC,prices
0,PET,FRE_000009,Marlin Estevi,PUR_00051-1,dog collar 20 cm color brown leather,collar,30,brown,MEX,7,6.53,18.81 | 24.85 | 22.99
1,PET,FRE_000009,Marlin Estevi,PUR_00051-2,dog collar 20 cm color black,collar,30,black,MEX,7,6.53,18.81 | 24.85 | 22.99
2,PET,FRE_000009,Marlin Estevi,PUR_00051-3,dog collar 20 cm color coffee,collar,30,coffee,MEX,6,6.53,18.81 | 24.85 | 22.99
3,PET,FRE_000009,Marlin Estevi,PUR_00051-4,dog collar 40 cm color brown leather,collar,40,brown,MEX,16,8.34,23.27 | 30.98 | 28.99
4,PET,FRE_000009,Marlin Estevi,PUR_00051-5,dog collar 40 cm color black,collar,40,black,MEX,15,8.34,23.27 | 30.98 | 28.99
5,PET,FRE_000009,Marlin Estevi,PUR_00051-6,dog collar 40 cm color coffee,collar,40,coffee,MEX,15,8.34,23.27 | 30.98 | 28.99
6,PET,FRE_000009,Marlin Estevi,PUR_00051-7,dog collar 50 cm color brown leather,collar,50,brown,MEX,10,12.75,34.17 | 45.97 | 35.99
7,PET,FRE_000009,Marlin Estevi,PUR_00051-8,dog collar 50 cm color black,collar,50,black,MEX,10,12.75,34.17 | 45.97 | 35.99
8,PET,FRE_000009,Marlin Estevi,PUR_00051-9,dog collar 50 cm color coffee,collar,50,coffee,MEX,10,12.75,34.17 | 45.97 | 35.99
9,KD,FRE_000009,Briseyda,PUR_00052-1,toy in felt dino long neck,toy,--,--,MEX,3,11.55,31.2 | 41.88 | 28.99


In [27]:
df_codes = df

In [28]:
for index,row in df_codes.iterrows():
    if not os.path.exists(row['freight_cod']):
        os.makedirs(row['freight_cod'])

    path = "{freight_cod}\\{identity_code}.jpeg".format(freight_cod=row['freight_cod'],identity_code=row['identity_code'])
    with open(path,"wb") as f:
        Code128(row['identity_code'], writer=ImageWriter()).write(f,options={"write_text": False})

In [29]:
df_catg = df[['freight_cod','Pctg']].drop_duplicates()
df_catg

,freight_cod,Pctg
0,FRE_000009,PET
9,FRE_000009,KD


In [30]:
from reportlab.pdfgen import canvas
import textwrap

for index,row in df_catg.iterrows():
    
    file = "{freight_cod}\\{Pctg}_booklet.pdf".format(freight_cod = row['freight_cod'],Pctg=row['Pctg'])
    pdf = canvas.Canvas(file)  

    df_code_ctg = df_codes[df_codes['Pctg']==row['Pctg']]

    y_start = 820
    pdf.setFontSize(10)
    pdf.drawString(20,y_start,'Pctg')
    pdf.drawString(45,y_start,'FC')
    pdf.drawString(85,y_start,'supplier')
    pdf.drawString(125,y_start,'IC')
    pdf.drawString(165,y_start,'descp.')
    pdf.drawString(235,y_start,'PC')
    pdf.drawString(265,y_start,'size')
    pdf.drawString(295,y_start,'clr')
    pdf.drawString(325,y_start,'Ctry')
    pdf.drawString(355,y_start,'units')
    pdf.drawString(385,y_start,'IC')
    pdf.drawString(415,y_start,'prices')
    pdf.drawString(475,y_start,'barcode')


    y_start = 800-25
    for index,row in df_code_ctg.iterrows():
        pdf.setFontSize(8)
        pdf.drawString(20,y_start,row['Pctg'])
        pdf.drawString(45,y_start,row['freight_cod'][4:])
        pdf.drawString(85,y_start,row['supplier'][0:7])
        pdf.drawString(125,y_start,row['identity_code'][4:])
        
        #description
        beginning_line = y_start
        textLines_lst = textwrap.wrap(row['description'], 15, break_long_words=False)
        for line in textLines_lst:
            pdf.drawString(165,beginning_line,line)
            beginning_line = beginning_line - 9

        pdf.drawString(235,y_start,row['PC'])
        pdf.drawString(265,y_start,row['size'])
        pdf.drawString(295,y_start,row['clr'])
        pdf.drawString(325,y_start,row['country'])
        pdf.drawString(355,y_start,str(row['units']))
        pdf.drawString(385,y_start,str(row['IC']))
        
        #prices
        beginning_line = y_start
        textLines_lst = textwrap.wrap(row['prices'], 15, break_long_words=False)
        for line in textLines_lst:
            pdf.drawString(415,beginning_line,line)
            beginning_line = beginning_line - 9

        #pdf.drawString(420,y_start,row['prices'])
        pdf.drawInlineImage('{freight_cod}\\{identity_code}.jpeg'.format(freight_cod = row['freight_cod'],identity_code = row['identity_code']),475,y_start-18, width=87*1.3,height=22*1.3)
        pdf.line(10,y_start+20,580,y_start+20)

        y_start = y_start - 45
        if y_start <=50:
            pdf.showPage()
            y_start = 820
            pdf.setFontSize(10)
            pdf.drawString(20,y_start,'Pctg')
            pdf.drawString(45,y_start,'FC')
            pdf.drawString(85,y_start,'supplier')
            pdf.drawString(125,y_start,'IC')
            pdf.drawString(165,y_start,'descp.')
            pdf.drawString(235,y_start,'PC')
            pdf.drawString(265,y_start,'size')
            pdf.drawString(295,y_start,'clr')
            pdf.drawString(325,y_start,'Ctry')
            pdf.drawString(355,y_start,'units')
            pdf.drawString(385,y_start,'IC')
            pdf.drawString(415,y_start,'prices')
            pdf.drawString(475,y_start,'barcode')
            y_start = 800-25
        else:
            pass
    pdf.save()
    
